## Injury classification with SageMaker

In [1]:
#https://wandb.ai/jack-morris/david-vs-goliath/reports/Does-Model-Size-Matter-A-Comparison-of-BERT-and-DistilBERT--VmlldzoxMDUxNzU
#https://towardsdatascience.com/working-with-hugging-face-transformers-and-tf-2-0-89bf35e3555a
#https://towardsdatascience.com/scale-neural-network-training-with-sagemaker-distributed-8cf3aefcff51
#https://towardsdatascience.com/how-to-reduce-training-time-for-a-deep-learning-model-using-tf-data-43e1989d2961
#https://towardsdatascience.com/hugging-face-transformers-fine-tuning-distilbert-for-binary-classification-tasks-490f1d192379
#https://towardsdatascience.com/to-distil-or-not-to-distil-bert-roberta-and-xlnet-c777ad92f8
#https://ymeadows.com/articles/fine-tuning-transformer-based-language-models

### Smart Batching
#https://towardsdatascience.com/divide-hugging-face-transformers-training-time-by-2-or-more-21bf7129db9q-21bf7129db9e
#https://www.youtube.com/watch?v=ynOZUNnbEWU

In [2]:
!python --version

Python 3.7.10


In [3]:
%%capture
!pip install tensorflow
!pip install transformers
!pip install nltk

In [4]:
import transformers
import nltk
print(nltk.__version__)

3.4.5


In [5]:
import nltk 

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import pandas as pd
import tensorflow as tf
import re
import nltk
import string
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras import activations, optimizers, losses
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import numpy as np
import sagemaker
from sagemaker import get_execution_role
import joblib 
import collections

In [7]:
print(tf.__version__)

2.5.0


In [8]:
bucket = 'cdc-cdh-sagemaker-s3fs-dev'
sagemaker_session = sagemaker.Session(default_bucket=bucket)
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()
default_bucket

'cdc-cdh-sagemaker-s3fs-dev'

In [7]:
%store -z
%store

Stored variables and their in-db values:


## Step 1 - Import and split data

In [8]:
def get_samples(ratio,train):
    great_than2_classes = train['event'].value_counts()[train['event'].value_counts() >2].index
    train = train[train['event'].isin(great_than2_classes.to_list())]
    train_samples, _ = train_test_split(train,train_size=ratio,random_state=42,stratify=train['event'])
    print("nb classes in samples",train_samples['event'].nunique())
    print("nb oservations:",train_samples.shape)
    #print(train_samples['event'].value_counts())

    return train_samples

In [9]:
def get_data(is_sample=None,ratio=None,is_test_split=False):
  
    train = pd.read_csv('train.csv')
    if is_sample:
        train = get_samples(ratio = ratio, train=train)
    great_than2_classes = train['event'].value_counts()[train['event'].value_counts() >5].index 
    train_filter = train[train['event'].isin(great_than2_classes.to_list())]
    
    print("nb classes in final data:",train_filter['event'].nunique())

    X = train_filter['text']
    y = train_filter['event']

    print(f"X.shape {X.shape} y.shape : {y.shape}")

    X_train_valid,X_test,y_train_valid, y_test = train_test_split(X,y,train_size=0.9,random_state=42,stratify=y)
    
    if is_test_split:
        X_train,X_valid,y_train,y_valid = train_test_split(X_train_valid,y_train_valid,train_size=0.8,random_state=42,stratify=y_train_valid)

    if is_test_split :
        print(f"X_train shape {X_train.shape} y_train shape : {y_train.shape}")
        print(f"X_valid shape {X_valid.shape} y_valid shape : {y_valid.shape}")
        print(f"X_test shape {X_test.shape} y_test shape : {y_test.shape}")
        
        return {
          'train': (X_train,y_train),
          'valid': (X_valid,y_valid),
          'test': (X_test,y_test)
      }

    else:
        print(f"X_train shape {X_train_valid.shape} y_train shape : {y_train_valid.shape}")
        print(f"X_valid shape {X_test.shape} y_valid shape : {y_test.shape}")
        
        return {
          'train': (X_train_valid,y_train_valid),
          'valid': (X_test,y_test),
      }


In [10]:
pd.options.display.max_colwidth = 3100
data = get_data(is_sample=False,ratio=1)
#data = get_data(is_test_split=True)
X_train, y_train = data['train']
X_valid,y_valid = data['valid']
#X_test,y_test = data['test']

nb classes in final data: 43
X.shape (153944,) y.shape : (153944,)
X_train shape (138549,) y_train shape : (138549,)
X_valid shape (15395,) y_valid shape : (15395,)


## Step 2 - Preprocess Data

In [11]:
import numpy as np
print('classes in train :',len(np.unique(y_train)))
print('classes in valid :',len(np.unique(y_valid)))

CLASSES = y_train.unique().tolist()
print(CLASSES)

classes in train : 43
classes in valid : 43
[62, 13, 71, 42, 60, 64, 55, 70, 31, 43, 63, 73, 24, 50, 53, 41, 66, 26, 78, 11, 99, 12, 65, 72, 23, 27, 44, 52, 69, 22, 51, 25, 32, 21, 40, 61, 56, 54, 49, 79, 67, 45, 20]


In [12]:
def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """

    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\"", "", text)

    # convert text to lowercase
    text = text.strip().lower()

    # remove all non-ASCII characters:
    text = re.sub(r'[^\x00-\x7f]', r'', text)

    # replace punctuation characters with spaces
    filters = '!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = " ".join(text.split())
    return text

In [13]:
def remove_useless_words(text,useless_words):
    sentence = [word for word in word_tokenize(text)]
    sentence_stop = [word for word in sentence if word not in useless_words]

    text = " ".join(sentence_stop)

    return text

In [14]:
def preprocess_data(X):
    """ Preprocess : cleaning and remove stop words"""

    X = X.apply(lambda x: re.sub(r'\d+', '', x))
    X = X.apply(lambda x: clean_text(x))

    stopwords = nltk.corpus.stopwords.words('english')
    useless_words = stopwords + list(string.punctuation) + ['yom', 'yof', 'yowm', 'yf', 'ym', 'yo']
    # print("useless word : ",useless_words)
    X = X.apply(lambda x: remove_useless_words(x,useless_words))

    return X

In [15]:
%%time
X_train_processed = preprocess_data(X_train)
X_valid_processed = preprocess_data(X_valid)

print("after preprocessing...")
print(X_train_processed.head(5))

after preprocessing...
69539                     work hammering metal pole hit finger dx r index finger pain communited intraarticular fx pip
76664     work pet groomer grooming dog another dog jumped upand dog bit pt r cheek abrasions jawline dx dog bite face
71708                                                                                     hit chisel eyebrown lac face
3165                      c upper arm shoulder pain w movement since yest think strained work lifting dx shoulder pain
136917                                                           fracture pubic rami semi load pvc pipe fell unloading
Name: text, dtype: object
CPU times: user 24.9 s, sys: 45.5 ms, total: 25 s
Wall time: 25 s


## Step 3 - Feature Extraction

### Tokenize and Encode Train and Test data

In [16]:
%%time
from transformers import  DistilBertTokenizerFast,BertTokenizerFast,RobertaTokenizerFast
MAX_LEN = 45

#MODEL_NAME = 'distilbert-base-uncased'
#tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)

MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

x_train = X_train_processed.to_list()
x_valid = X_valid_processed.to_list()


train_encodings = tokenizer(x_train, max_length=MAX_LEN, truncation=True, padding='max_length',return_tensors='tf')
valid_encodings = tokenizer(x_valid, max_length=MAX_LEN, truncation=True, padding='max_length',return_tensors='tf')


CPU times: user 16.4 s, sys: 1.73 s, total: 18.1 s
Wall time: 6.46 s


In [17]:
print(train_encodings['input_ids'][0])
print(train_encodings['attention_mask'][0])

tf.Tensor(
[  101  2147 27883  3384  6536  2718  4344  1040  2595  1054  5950  4344
  3255  4012 23041 17572 26721  8445 21412 23292 28315   102     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0], shape=(45,), dtype=int32)
tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0], shape=(45,), dtype=int32)


### Encoding labels

In [18]:
from sklearn.preprocessing import LabelEncoder

print("Encoding Labels .....")
encoder = LabelEncoder()
encoder.fit(y_train)
y_train_encode = np.asarray(encoder.transform(y_train))
y_valid_encode = np.asarray(encoder.transform(y_valid))

Encoding Labels .....


In [19]:
id2label = { id:str(label) for id, label in enumerate(encoder.classes_)}
label2id = { str(label):id for id, label in enumerate(encoder.classes_)}

In [20]:
print('classes in train :',len(np.unique(y_train_encode)))
print('classes in valid :',len(np.unique(y_valid_encode)))

classes in train : 43
classes in valid : 43


In [21]:
import os
data_path='./data'
os.makedirs(data_path,exist_ok=True)

from pickle import dump
dump(encoder,open(os.path.join(data_path,'encode.pkl'),'wb'))

### Create TF Dataset

In [22]:
%%time
def construct_tfdataset(encodings, y=None):
    if y is not None:
        return tf.data.Dataset.from_tensor_slices((dict(encodings),y))
    else:
        # this case is used when making predictions on unseen samples after training
        return tf.data.Dataset.from_tensor_slices(dict(encodings))
    
train_tfdataset = construct_tfdataset(train_encodings, y_train_encode)
valid_tfdataset = construct_tfdataset(valid_encodings, y_valid_encode)

CPU times: user 3.81 ms, sys: 0 ns, total: 3.81 ms
Wall time: 3.67 ms


### Create TF Records File

In [23]:

def _save_feature_as_tfrecord(tfdataset,file_path):
    """ Helper function to save the tf dataset as tfrecords"""
    

    def single_example_data(encoding,label_id):

        input_ids =encoding['input_ids']
        attention_mask = encoding['attention_mask']

        tfrecord_features = collections.OrderedDict()

        def _int64_feature(value):
            """Returns an int64_list from a bool / enum / int / uint."""
            return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

        tfrecord_features['input_ids'] = _int64_feature(input_ids)
        tfrecord_features['attention_mask'] = _int64_feature(attention_mask)
        tfrecord_features['label_ids'] =  _int64_feature([label_id])

        _example = tf.train.Example(features=tf.train.Features(feature=tfrecord_features))

        return _example.SerializeToString()

    def data_generator():
        for features in tfdataset:
            yield single_example_data(*features)

    serialized_tfdataset = tf.data.Dataset.from_generator(
        data_generator, output_types=tf.string, output_shapes=())        

    writer = tf.data.experimental.TFRecordWriter(file_path)
    writer.write(serialized_tfdataset)


        

In [24]:
%time

# training data
_save_feature_as_tfrecord(train_tfdataset,'./data/train.tfrecord')

#validation data
_save_feature_as_tfrecord(valid_tfdataset,'./data/valid.tfrecord')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


### Upload data to S3

In [25]:
prefix = 'projects/project006/injury-data'
tfrecord_data_location = sagemaker_session.upload_data(path = './data',
                                                      bucket = bucket,
                                                      key_prefix = prefix)
tfrecord_data_location

's3://cdc-cdh-sagemaker-s3fs-dev/projects/project006/injury-data'

## Step 4 - Run DistilBERT Training Job

### Training output path

In [26]:
prefix = 'projects/project006/output'
output_path = 's3://{}/{}/'.format(bucket, prefix )
output_path

's3://cdc-cdh-sagemaker-s3fs-dev/projects/project006/output/'

### Training script

In [27]:
# TensorFlow 2.3 script
!pygmentize './src/train.py'

import os
import tensorflow as tf
import numpy as np
import argparse
from transformers import  TFDistilBertForSequenceClassification,DistilBertConfig
from transformers import TFBertForSequenceClassification, BertConfig
from transformers import TFRobertaForSequenceClassification, RobertaConfig
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from transformers.optimization_tf import WarmUp, AdamWeightDecay
from pickle import load

def _parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model-dir",type=str)
    parser.add_argument('--output-data-dir', type=str,default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument("--sm-model-dir",type=str,default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train",type=str,default=os.environ.get("SM_CHANNEL_TRAIN"))
    
    ## Hyperparams
    parser.add_argument("--model_name",type=str,default='DistilBERT')
    parser.add_argument("--num_records",type=int)
    parser.add_argument("--

### Hyperparameters

In [28]:
num_records = len(X_train)
num_valid_records = len(X_valid)
max_len = MAX_LEN
epochs = 5
batch_size = 16
valid_batch_size = 16
steps_per_epoch = num_records // batch_size
validation_steps = num_valid_records // valid_batch_size
learning_rate = 5e-5
optimizer = 'adam'

### Using Hugging Face Estimator to start training

In [37]:
import time
from time import gmtime, strftime
from sagemaker.huggingface import HuggingFace

model_name = 'DistilBERT'
job_name_prefix = f"training-{model_name}"
timestamp = strftime("-%m-%d-%M-%S", gmtime())

job_name = job_name_prefix + timestamp

_distillbert_estimator = HuggingFace(
        base_job_name  = job_name,
        entry_point="train.py",
        source_dir = "./src/",
        role=role,
        instance_count=1,
        volume_size = 5,
        max_run = 18000,
        instance_type='ml.p3.2xlarge',
        transformers_version = "4.4",
        tensorflow_version  = "2.4",
        py_version="py37",
        output_path = output_path,
        hyperparameters = {
                "model_name": model_name,
                "num_records":  num_records,
                "max_len":max_len,
                "epochs":int(epochs),
                "learning_rate":float(learning_rate),
                "batch_size":int(batch_size),
                "valid_batch_size":valid_batch_size,
                "steps_per_epoch": steps_per_epoch,
                "validation_steps": validation_steps,
                "optimizer":optimizer
                },
        metric_definitions = [{'Name':'train:loss','Regex':'loss: ([0-9\\.]+)'},
                                    {'Name':'train:accuracy','Regex':'acc: ([0-9\\.]+)'},
                                    {'Name':'validation:loss','Regex':'val_loss: ([0-9\\.]+)'},
                                    {'Name':'validation:accuracy','Regex':'val_acc: ([0-9\\.]+)'}],
        enable_sagemaker_metrics = True
    )

_distillbert_estimator.fit({'train':tfrecord_data_location}, wait=False)

### Download the trained distilbert model

In [ ]:
job_name = ''

def download_model(sagemaker_session,job_name,estimator = None):
    
    if job_name is not None :
        modeldesc = sagemaker_session.describe_training_job(job_name)
        s3_model_path = modeldesc['ModelArtifacts']['S3ModelArtifacts']
    elif estimator is not None:
        s3_model_path =  _distillbert_estimator.model_data

    
    os.makedirs(f"./output/model/{job_name}/",exist_ok=True)

    S3Downloader.download(
        s3_uri=s3_model_path, # s3 uri where the trained model is located
        local_path=f"./output/model/{job_name}/", # local path where *.targ.gz is saved
        sagemaker_session=sagemaker_session # sagemaker session used for training the model
    )
    
    return s3_model_path, modeldesc['HyperParameters']




In [ ]:
#downloading model
s3_model_path, hp = download_model(sagemaker_session,job_name)

print(s3_model_path)
print(hp)

### Extract and load the model

In [ ]:
import tarfile
from transformers.optimization_tf import AdamWeightDecay
from tensorflow.keras.optimizers.schedules import PolynomialDecay

def extract_data_load_model(job_name,model_name):

    t = tarfile.open(f'./output/model/{job_name}/model.tar.gz', 'r:gz')
    t.extractall(path=f'./output/model/{job_name}')
    _model = tf.keras.models.load_model(f"./output/model/{job_name}/{model_name}",custom_objects={'AdamWeightDecay':AdamWeightDecay})
    
    return _model

In [39]:
model_name = 'DistilBERT'
loaded_model = extract_data_load_model(job_name,model_name)


## Step 5 - Evaluate the model

### Load training and validation data

In [40]:
def _load_data(train_dir,MAX_LEN,epochs,batch_size,valid_batch_size,steps_per_epoch,validation_steps):
    """ Helper function to load,parse and create input data pipeline from TFRecords"""
          
    train_file = os.path.join(train_dir,"train.tfrecord") 
    valid_file = os.path.join(train_dir,"valid.tfrecord")
    
    # Create a description of the features.
    feature_description = {
        'input_ids': tf.io.FixedLenFeature([MAX_LEN], tf.int64),
        'attention_mask': tf.io.FixedLenFeature([MAX_LEN], tf.int64),
        'label_ids': tf.io.FixedLenFeature([], tf.int64),
    }
        
    def _parse_function(example_proto):

        # Parse the input `tf.train.Example` proto using the dictionary above.
        parsed  = tf.io.parse_single_example(example_proto, feature_description)

        return {'input_ids':parsed['input_ids'],'attention_mask':parsed['attention_mask']},parsed['label_ids']
        
    
    train_dataset = tf.data.TFRecordDataset(train_file)
    train_dataset = train_dataset.repeat(epochs * steps_per_epoch)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    train_dataset = train_dataset.map(_parse_function,num_parallel_calls=tf.data.AUTOTUNE)
    
    train_dataset = train_dataset.batch(batch_size)
    
    
    
    valid_dataset = tf.data.TFRecordDataset(valid_file)
    valid_dataset = valid_dataset.repeat(epochs * validation_steps)
    valid_dataset = valid_dataset.prefetch(tf.data.AUTOTUNE)
    valid_dataset = valid_dataset.map(_parse_function,num_parallel_calls=tf.data.AUTOTUNE)
    valid_dataset = valid_dataset.batch(valid_batch_size)
    
   
    return train_dataset, valid_dataset

In [41]:
train_dataset,valid_dataset = _load_data('./data',MAX_LEN,epochs,batch_size,valid_batch_size,steps_per_epoch,validation_steps)

### Evaluate the model

In [42]:

def _evaluate_model(loaded_model):
    print("Evaluating Training data...")
    train_score = loaded_model.evaluate(train_dataset,
                                     steps = steps_per_epoch,
                                     batch_size=batch_size)

    print("Training loss: ", train_score[0])
    print("Training accuracy: ", train_score[1])

    print("Evaluating Validation data...")
    valid_score = loaded_model.evaluate(valid_dataset, steps = validation_steps,batch_size=valid_batch_size)
    print("Validation loss: ", valid_score[0])
    print("Validation accuracy: ", valid_score[1])
    
_evaluate_model(loaded_model)

Evaluating Training data...
8659/8659 [==============================] - 6041s 697ms/step - loss: 0.1104 - acc: 0.9659
Training loss:  0.11043110489845276
Training accuracy:  0.9659242033958435
Evaluating Validation data...
962/962 [==============================] - 673s 699ms/step - loss: 0.5459 - acc: 0.8493
Validation loss:  0.5459089875221252
Validation accuracy:  0.8492723703384399


### Using the fine-tuned model to predict new samples

In [83]:
MODEL_NAME = 'distilbert-base-uncased'

def distilBert_create_predictor(model, encoder,model_name, max_len,text):
    tkzr = DistilBertTokenizerFast.from_pretrained(model_name)
    x = [text]
    encodings =  tkzr(x, max_length=max_len, truncation=True, padding='max_length',return_tensors='tf')
    tfdataset = construct_tfdataset(encodings)
    tfdataset = tfdataset.batch(1)
    preds = model.predict(tfdataset)
    categories = encoder.classes_.tolist()
    enc = np.argmax(preds[0])
    
    return {     'text' : x,
                 'predict_proba' : preds[0][np.argmax(preds[0])],
                 'predicted_class' : categories[np.argmax(preds)]                             
                }

In [84]:
x = 'sp lt ankle going steps twisted ankle work'
distilBert_create_predictor(loaded_model, encoder,MODEL_NAME, MAX_LEN,x)

{'text': ['sp lt ankle going steps twisted ankle work'],
 'predict_proba': 0.9913174,
 'predicted_class': 73}

## Step 6 - Save Predictions and calculate metrics

In [85]:
%%time
import numpy as np

model_name = 'distilbert-base-uncased'
x = X_valid_processed.tolist()
y = y_valid.tolist()

def distilbert_batch_predict(model, encoder,model_name, max_len,x,y) :
    tkzr = DistilBertTokenizerFast.from_pretrained(model_name)
    encodings_x =  tkzr(x, max_length=MAX_LEN, truncation=True, padding='max_length',return_tensors='tf')
    tfdataset = construct_tfdataset(encodings_x).batch(32)
    preds = loaded_model.predict(tfdataset)
    predictions_encode = pd.DataFrame(data=preds).apply(lambda x: np.argmax(x),axis=1)
    categories = encoder.classes_.tolist()
    predictions_event= predictions_encode.apply(lambda x:categories[x])


    print(len(predictions_event))
    results = pd.DataFrame({'text': x,
                      'true_event':y,
                      'preds_encode': predictions_encode,
                      'preds_event':predictions_event
                            })


    return results

results = distilbert_batch_predict(loaded_model,encoder,model_name,MAX_LEN,x,y)
results.to_csv('solution.csv')
results.head(10)

1534
CPU times: user 1min 39s, sys: 4.6 s, total: 1min 44s
Wall time: 16.9 s


text  \
0                 r hand pain c r hand pain rest denies injury overuse pushing carts local grocery store work   
1                                                         transferring patient fell landing floor back sprain   
2                        head cont work slipped threshold door falling backward striking back head denies loc   
3                                                                                    contusion hand fell work   
4                               trying flush iv work became clogged sprayed pt face dx exposure bodily fluids   
5                                                  dx forearm elbow laceration p sustained laceration working   
6                                    subject officer c rt nd finger pain altercation w persondx finger strain   
7                                                         burn r index finger w hot oil work x nd degree burn   
8  l forearm lac walking w sharp metal place arm sharp edge several minutes moved arm noted gaping laceration   
9                                                                                    strained lower back work   

   true_event  preds_encode  preds_event  
0          71            23           71  
1          12            10           42  
2          42            10           42  
3          42            10           42  
4          55            16           55  
5          60            17           60  
6          11             0           11  
7          53            15           53  
8          60            19           63  
9          70            22           70

### Calculate metrics

In [86]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,balanced_accuracy_score
import pandas as pd

def compute_metrics(pred):
    labels = pred.true_event
    preds = pred.preds_event
    acc = accuracy_score(labels, preds)
    bal_acc = balanced_accuracy_score(labels, preds)
    precision = precision_score(labels,preds,average='macro')
    recall = recall_score(labels,preds,average='macro')
    f1 = f1_score(labels,preds,average='macro')
    return {
        'accuracy': acc,
        'balanced_accuracy':bal_acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

compute_metrics(results)

{'accuracy': 0.7946544980443285,
 'balanced_accuracy': 0.697549854765685,
 'f1': 0.7125622769786281,
 'precision': 0.7423009184922906,
 'recall': 0.697549854765685}

## Hyperparameter Tuning (HPT)

In [40]:
tuning_job_name_prefix = f"tuning-{model_name}"
timestamp = strftime("-%m-%d-%M-%S", gmtime())
base_tuning_job_name = tuning_job_name_prefix + timestamp
base_tuning_job_name

'tuning-DistilBERT-07-28-27-28'

In [42]:
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter,CategoricalParameter

# Configure HyperparameterTuner
_tuner = HyperparameterTuner(
                             base_tuning_job_name = base_tuning_job_name,
                             estimator=_distillbert_estimator,  # previously-configured Estimator object
                             objective_metric_name='validation:accuracy',
                             hyperparameter_ranges={'learning_rate': CategoricalParameter(['2e-5','3e-5', '5e-5']),
                                                      'epochs':CategoricalParameter(['4','5']),
                                                      'batch_size':CategoricalParameter(['16','32']) 
                                                   },
                             metric_definitions = [{'Name':'validation:accuracy','Regex':'val_acc: ([0-9\\.]+)'}],
                             max_jobs=12,
                             max_parallel_jobs=2)
    
   
    
print(job_name,base_tuning_job_name)
_tuner.fit({'train':tfrecord_data_location}, wait=False)

training-DistilBERT-07-28-27-19 tuning-DistilBERT-07-28-27-28


### Base-BERT

In [29]:
import time
from time import gmtime, strftime
from sagemaker.huggingface import HuggingFace

model_name = 'BaseBERT'
job_name_prefix = f"training-{model_name}"
timestamp = strftime("-%m-%d-%M-%S", gmtime())

job_name = job_name_prefix + timestamp

_bert_estimator = HuggingFace(
        base_job_name  = job_name,
        entry_point="train.py",
        source_dir = "./src/",
        role=role,
        instance_count=1,
        volume_size = 5,
        max_run = 18000,
        instance_type='ml.p3.2xlarge',
        transformers_version = "4.4",
        tensorflow_version  = "2.4",
        py_version="py37",
        output_path = output_path,
        hyperparameters = {
                "model_name": model_name,
                "num_records":  num_records,
                "max_len":max_len,
                "epochs":int(epochs),
                "learning_rate":float(learning_rate),
                "batch_size":int(batch_size),
                "valid_batch_size":valid_batch_size,
                "steps_per_epoch": steps_per_epoch,
                "validation_steps": validation_steps,
                "optimizer":optimizer
                },
        metric_definitions = [{'Name':'train:loss','Regex':'loss: ([0-9\\.]+)'},
                                    {'Name':'train:accuracy','Regex':'acc: ([0-9\\.]+)'},
                                    {'Name':'validation:loss','Regex':'val_loss: ([0-9\\.]+)'},
                                    {'Name':'validation:accuracy','Regex':'val_acc: ([0-9\\.]+)'}],
        enable_sagemaker_metrics = True
    )

_bert_estimator.fit({'train':tfrecord_data_location}, wait=False)

In [35]:
from sagemaker.s3 import S3Downloader

S3Downloader.download(
    s3_uri=_bert_estimator.model_data, # s3 uri where the trained model is located
    local_path='./output/model/basebert/', # local path where *.targ.gz is saved
    sagemaker_session=sagemaker_session # sagemaker session used for training the model
)

In [37]:
import tarfile
from transformers.optimization_tf import AdamWeightDecay
from tensorflow.keras.optimizers.schedules import PolynomialDecay


t = tarfile.open('./output/model/basebert/model.tar.gz', 'r:gz')
t.extractall(path='./output/model/basebert')
bert_model = tf.keras.models.load_model('./output/model/basebert/BaseBERT',custom_objects={'AdamWeightDecay':AdamWeightDecay})

In [42]:
def _evaluate_model(loaded_model):
    print("Evaluating Training data...")
    train_score = loaded_model.evaluate(train_dataset,
                                     steps = steps_per_epoch,
                                     batch_size=batch_size)

    print("Training loss: ", train_score[0])
    print("Training accuracy: ", train_score[1])

    print("Evaluating Validation data...")
    valid_score = loaded_model.evaluate(valid_dataset, steps = validation_steps,batch_size=valid_batch_size)
    print("Validation loss: ", valid_score[0])
    print("Validation accuracy: ", valid_score[1])
    
_evaluate_model(bert_model)

Evaluating Training data...
8659/8659 [==============================] - 3195s 369ms/step - loss: 0.1164 - acc: 0.9641
Training loss:  0.11638274043798447
Training accuracy:  0.9641124606132507
Evaluating Validation data...
962/962 [==============================] - 355s 368ms/step - loss: 0.5353 - acc: 0.8526
Validation loss:  0.535329282283783
Validation accuracy:  0.8525857329368591
